In [6]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import glob as glob
import cv2
import sys
from os import listdir
from os.path import isfile, join
import time
from PIL import ImageFile
import shutil
%matplotlib inline
sys.path.append("..")
CWD_PATH = os.getcwd()

In [7]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [8]:
MODEL_NAME = 'ssd_mobilenet_v2_german'
#MODEL_NAME = 'ssd_mobilenet_v1'
#MODEL_NAME = 'faster_rcnn_inception_v2'
#MODEL_NAME = 'ssd_inception_v2'
#MODEL_NAME = 'faster_rcnn_resnet50'
#MODEL_NAME = 'faster_rcnn_resnet_101'

In [9]:
MODEL_PATH = os.path.join('model', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'weights','saved_model')
#PATH_TO_CKPT = os.path.join(MODEL_PATH,'saved_model')
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('new_stop_sign_data', 'stop_label_map.pbtxt')

NUM_CLASSES = 9

In [10]:
print('PATH_TO_CKPT :', PATH_TO_CKPT)
print('PATH_TO_LABELS :', PATH_TO_LABELS)
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load(PATH_TO_CKPT)

PATH_TO_CKPT : model/ssd_mobilenet_v2_german/weights/saved_model
PATH_TO_LABELS : new_stop_sign_data/stop_label_map.pbtxt


In [11]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
print(label_map)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
print(categories)
category_index = label_map_util.create_category_index(categories)
print(category_index)

item {
  name: "stop"
  id: 1
}
item {
  name: "speed_limit_20"
  id: 2
}
item {
  name: "speed_limit_30"
  id: 3
}
item {
  name: "speed_limit_50"
  id: 4
}
item {
  name: "speed_limit_60"
  id: 5
}
item {
  name: "speed_limit_70"
  id: 6
}
item {
  name: "speed_limit_80"
  id: 7
}
item {
  name: "speed_limit_100"
  id: 8
}
item {
  name: "speed_limit_120"
  id: 9
}

[{'id': 1, 'name': 'stop'}, {'id': 2, 'name': 'speed_limit_20'}, {'id': 3, 'name': 'speed_limit_30'}, {'id': 4, 'name': 'speed_limit_50'}, {'id': 5, 'name': 'speed_limit_60'}, {'id': 6, 'name': 'speed_limit_70'}, {'id': 7, 'name': 'speed_limit_80'}, {'id': 8, 'name': 'speed_limit_100'}, {'id': 9, 'name': 'speed_limit_120'}]
{1: {'id': 1, 'name': 'stop'}, 2: {'id': 2, 'name': 'speed_limit_20'}, 3: {'id': 3, 'name': 'speed_limit_30'}, 4: {'id': 4, 'name': 'speed_limit_50'}, 5: {'id': 5, 'name': 'speed_limit_60'}, 6: {'id': 6, 'name': 'speed_limit_70'}, 7: {'id': 7, 'name': 'speed_limit_80'}, 8: {'id': 8, 'name': 'speed_limi

In [12]:
PATH_TO_TEST_IMAGES_DIR = 'new_stop_sign_data/images/test'
image_name=[]

In [13]:
from natsort import natsorted
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, f) for f in listdir(PATH_TO_TEST_IMAGES_DIR) if isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, f))]
TEST_IMAGE_PATHS = sorted(TEST_IMAGE_PATHS)
TEST_IMAGE_PATHS = natsorted(TEST_IMAGE_PATHS)
#print('TEST_IMAGE_PATHS :',TEST_IMAGE_PATHS)

In [14]:
image_name=[f for f in listdir(PATH_TO_TEST_IMAGES_DIR) if isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, f))]
image_name = sorted(image_name)
image_name = natsorted(image_name)
print(image_name)

['00056.jpg', '00125.jpg', '00134.jpg', '00167.jpg', '00174.jpg', '00181.jpg', '00182.jpg', '00192.jpg', '00223.jpg', '00259.jpg', '00260.jpg', '00285.jpg', '00300.jpg', '00303.jpg', '00316.jpg', '00322.jpg', '00334.jpg', '00348.jpg', '00369.jpg', '00377.jpg', '00405.jpg', '00418.jpg', '00439.jpg', '00441.jpg', '00444.jpg', '00456.jpg', '00457.jpg', '00459.jpg', '00477.jpg', '00480.jpg', '00496.jpg', '00512.jpg', '00513.jpg', '00518.jpg', '00519.jpg', '00524.jpg', '00528.jpg', '00605.jpg', '00609.jpg', '00613.jpg', '00632.jpg', '00651.jpg', '00661.jpg', '00667.jpg', '00670.jpg', '00674.jpg', '00680.jpg', '00688.jpg', '00690.jpg', '00699.jpg', '00701.jpg', '00706.jpg', '00707.jpg', '00716.jpg', '00729.jpg', '00730.jpg', '00734.jpg', '00740.jpg', '00758.jpg', '00789.jpg', '00806.jpg', '00809.jpg', '00829.jpg', '00834.jpg', '00839.jpg', '00849.jpg', '00850.jpg', '00854.jpg', '00860.jpg', '00881.jpg', '00884.jpg', '00891.jpg']


In [16]:
#def DetectImagesFromFolder(detector, images_dir, save_output=False, output_dir='output/'):

for file in os.scandir(PATH_TO_TEST_IMAGES_DIR):
    if file.is_file() and file.name.endswith(('.jpg', '.jpeg', '.png')) :
        image_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, file.name)
        #class_id = 1
        print(image_path)
        img = cv2.imread(image_path)
        image_np1 = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        #image_np1 = np.expand_dims(image_np1, axis=0)
        #Threshold = 0.5
        #print('img : ', img)
        #det_boxes = DetectFromImage(img)
        #img = detector.DisplayDetections(img, det_boxes)
        #img_out = os.path.join(output_dir, file.name)
        #cv2.imwrite(img_out, img)
        #im_height, im_width, _ = img.shape
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        #input_tensor = np.array(img.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
        input_tensor = np.expand_dims(img, 0)
        detections = detect_fn(input_tensor)

        bboxes = detections['detection_boxes'][0].numpy()
        bclasses = detections['detection_classes'][0].numpy().astype(np.int32)
        bscores = detections['detection_scores'][0].numpy()
        #det_boxes = ExtractBBoxes(bboxes, bclasses, bscores, im_width, im_height)
        #print('bclasses : ', bclasses)
        IMAGE_SIZE = (20, 20)
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np1,
            np.squeeze(bboxes),
            np.squeeze(bclasses).astype(np.int32),
            np.squeeze(bscores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=4,
            min_score_thresh=0.4)
        #print('img1 :', img1)
        im = Image.fromarray(image_np1)
        #img_out = os.path.join('output_1', file.name)
        #print('img_out :', img_out)
        #cv2.imwrite(img_out, im)
        im.save(os.path.join(CWD_PATH, 'output_stop_and_speed_limit',file.name))

new_stop_sign_data/images/test/00480.jpg
new_stop_sign_data/images/test/00441.jpg
new_stop_sign_data/images/test/00891.jpg
new_stop_sign_data/images/test/00456.jpg
new_stop_sign_data/images/test/00300.jpg
new_stop_sign_data/images/test/00734.jpg
new_stop_sign_data/images/test/00405.jpg
new_stop_sign_data/images/test/00740.jpg
new_stop_sign_data/images/test/00519.jpg
new_stop_sign_data/images/test/00834.jpg
new_stop_sign_data/images/test/00609.jpg
new_stop_sign_data/images/test/00806.jpg
new_stop_sign_data/images/test/00670.jpg
new_stop_sign_data/images/test/00661.jpg
new_stop_sign_data/images/test/00860.jpg
new_stop_sign_data/images/test/00056.jpg
new_stop_sign_data/images/test/00418.jpg
new_stop_sign_data/images/test/00674.jpg
new_stop_sign_data/images/test/00758.jpg
new_stop_sign_data/images/test/00706.jpg
new_stop_sign_data/images/test/00192.jpg
new_stop_sign_data/images/test/00690.jpg
new_stop_sign_data/images/test/00316.jpg
new_stop_sign_data/images/test/00528.jpg
new_stop_sign_da